# Orbits of bitangents on symmetric quartics

## Table of contents
1. [Introduction](#introduction)
3. [Type I: The Klein ($\text{PSL}_2(7)$) quartic](#klein)
4. [Type II: The Clebsch](#fermat)
5. [Type III: The (48,33) quartic](#type3)
6. [Type IV: The $S_4$ quartic](#S4)
7. [Type V: The (16,13) quartic](#1613)
8. [Type VI: The $C_9$ quartic](#C9)
9. [Type VII: The $D_8$ quartic](#D8)
10. [Type IX: The $S_3$ quartic](#S3)
11. [Type X: The $K_4$ quartic](#K4)
12. [Type XI: The $C_3$ quartic](#C3)
13. [Type XII: The $C_2$ quartic](#C2)

# Introduction<a name="introduction"></a>
Here we provide some code which can be used to study bitangents for example quartics of each type. For the first few types we explain in great detail how the code is working, then omit detail as the pattern becomes clear. To investigate orbits in each case, we:
1. Pick an example quartic in that fmaily
2. Compute its bitangents numerically
3. Plug in its group of symmetries $G\le \text{PGL}_3(\mathbb{C})$ as a matrix group, and use this to act on the bitangents
4. Group the bitangents into orbits
5. Determine isotropy for each orbit

## Imports
The following code is taken from [Vinzant's webpage](http://sites.math.washington.edu/~vinzant/research/quartics/quartictype.sage) from [her research on quartics](http://sites.math.washington.edu/~vinzant/research/quartics.html) with Plaumann and Sturmfels. It inputs a quartic polynomial and outputs (among other things) equations for its tritangents.

In [1]:
%run compute_bitangents.ipynb

We import some helper functions which let us investigate group actions on bitangents numerically

In [2]:
%run orbit_isotropy_functions.ipynb
F=QQ; T.<x,y,z>=PolynomialRing(F,order='degrevlex')

# Type I: The Klein quartic<a name="klein"></a>
Our first quartic is the *Klein quartic*, defined by the equation
$$x^3y + y^3z + z^3x$$
It is a classical fact that the automorphism group of the Klein quartic acts transitively on its 28 bitangents, however we can also observe this computationally.

## The Automorphism group of the Klein
This one is the most painful to encode due to some of the strange elements appearing in a presentaion of it as a subgroup of $\text{PGL}_3(\mathbb{C})$, for instance:
$$\alpha = \frac{\zeta_7^5 - \zeta_7^2}{\sqrt{-7}},$$
where $\zeta_7$ is a primitive 7th root of unity. We could naively code this up, however Sage will interpret primitive roots of unity and square roots jointly as living inside of $\mathbb{C}$. Instead we should exploit the Kronecker-Weber theorem, and rewrite $\sqrt{-7}$ as follows:
$$\sqrt{-7} = 1 + 2\zeta_7 + 2\zeta_7^2 + 2\zeta_7^4.$$

In [3]:
sqrneg7 = 1 + 2*E(7) + 2*E(7)^2 + 2*E(7)^4;
alpha = (E(7)^5 - E(7)^2)/sqrneg7
beta = (E(7)^6 - E(7))/sqrneg7
gamma = (E(7)^3 - E(7)^4)/sqrneg7

We define the generators. The variable names for generators are consistent across groups -- e.g. the $2$nd generator of the Type $7$ quartic for instance will be `G7gen2`.

In [4]:
G1gen1 = matrix([
    [E(7)^4,0,0],
    [0,E(7)^2,0],
    [0,0,E(7)]
])
G1gen2 = matrix([
    [0,1,0],
    [0,0,1],
    [1,0,0]
])
G1gen3 = matrix([
    [beta,alpha,gamma],
    [alpha,gamma,beta],
    [gamma,beta,alpha]
])

G1 = MatrixGroup([G1gen1,G1gen2,G1gen3])

This `G1` is the automorphism group of the Klein quartic. If we want to double check the GAP ID of this small group, we can do so by using the [Sage interface to GAP](https://doc.sagemath.org/html/en/reference/interfaces/sage/interfaces/gap.html), for instance:

In [5]:
get_gap_id(G1)

'[ 168, 42 ]'

## Transitivity of the automorphism group of the Klein quartic on its bitangents

To compute the bitangents numerically and group them into orbits, we run our method `orbits_of_bitangents`:

In [6]:
Klein = x^3*y + y^3*z + z^3*x
orbits_of_bitangents(Klein,G1)

[[x + 0.308*y + 1.555*z,
  x + 5.0489*y + 3.2471*z,
  x + (-0.0685 + 0.3003*I)*y + (-1.401 + 0.6747*I)*z,
  x + (-1.1239 - 4.9223*I)*y + (-2.9256 - 1.4086*I)*z,
  x + (0.6235 - 0.7818*I)*y + (-0.2226 + 0.9749*I)*z,
  x + (-0.2775 + 0.1336*I)*y + (0.9695 + 1.2158*I)*z,
  x + (-0.2226 + 0.9749*I)*y + (-0.901 + 0.4339*I)*z,
  x + (0.401 + 0.5028*I)*y + (-0.0441 - 0.1931*I)*z,
  x + (-1.1235 + 4.9224*I)*y + (-2.9255 + 1.4089*I)*z,
  x + (-0.2775 - 0.1336*I)*y + (0.9695 - 1.2158*I)*z,
  x + (0.6235 + 0.7818*I)*y + (-0.2225 - 0.9749*I)*z,
  x + (-0.0685 - 0.3003*I)*y + (-1.401 - 0.6747*I)*z,
  x + (-0.901 - 0.4339*I)*y + (0.6235 - 0.7818*I)*z,
  x + (-0.5794 + 0.279*I)*y + (0.1235 + 0.1548*I)*z,
  x + (-4.5489 - 2.1907*I)*y + (2.0245 - 2.5387*I)*z,
  x + (0.192 - 0.2408*I)*y + (-0.346 + 1.516*I)*z,
  x + (-0.901 + 0.4339*I)*y + (0.6235 + 0.7819*I)*z,
  x + (-0.1431 - 0.627*I)*y + (-0.1785 - 0.0859*I)*z,
  x + (3.148 - 3.9474*I)*y + (-0.7225 + 3.1657*I)*z,
  x + (0.192 + 0.2408*I)*y + (-0.346

This has one orbit, as we see above. Hence the group acts transitively on all 28 bitangents, which was known to Klein. If we wanted to recover the isotropy of this orbit (`orbits_of_bitangents(Klein,G1)[0]`) we could run our isotropy method:

In [7]:
Klein_isotropy = isotropy(orbits_of_bitangents(Klein,G1)[0],G1)

By checking its GAP ID for instance we see it is the symmetric group $S_3$:

In [8]:
get_gap_id(Klein_isotropy)

'[ 6, 1 ]'

Therefore as a $\text{PSL}_2(7)$-set, the 28 bitangents of the Klein quartic are in equivariant bijection with the left cosets $[\text{PSL}_2(7)/S_3]$. As a sanity check, $|\text{PSL}_2(7)| = 168$ and $|S_3| = 6$, and $168/6 = 28$.

# Type II: Fermat<a href="fermat"></a>
The Fermat is defined as follows:

In [9]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
Fermat = x^4 + y^4 + z^4

Its automorphism group is order 96, given as follows:

In [10]:
G2gen1=matrix([[0,0,1],[1,0,0],[0,1,0]])
G2gen2=matrix([[-E(4),0,0],[0,0,1],[0,E(4),0]])
G2=MatrixGroup(G2gen1,G2gen2)

We investigate the orbits of the action on the bitangents as follows:

In [11]:
Fermat_orbits = orbits_of_bitangents(Fermat,G2);Fermat_orbits

[[y + (-0.7071 - 0.7071*I)*z,
  y + (0.7071 + 0.7071*I)*z,
  y + (0.7071 - 0.7071*I)*z,
  y + (-0.7071 + 0.7071*I)*z,
  x + (-0.7071 - 0.7071*I)*y,
  x + (0.7071 + 0.7071*I)*y,
  x + (0.7071 - 0.7071*I)*y,
  x + (-0.7071 + 0.7071*I)*y,
  x + (-0.7071 + 0.7071*I)*z,
  x + (0.7071 - 0.7071*I)*z,
  x + (0.7071 + 0.7071*I)*z,
  x + (-0.7071 - 0.7071*I)*z],
 [x + y - z,
  x + y + z,
  x - y - z,
  x - y + z,
  x - y + 1.0*I*z,
  x - y + (-1.0*I)*z,
  x + y + 1.0*I*z,
  x + y + (-1.0*I)*z,
  x + 1.0*I*y + z,
  x + 1.0*I*y - z,
  x + (-1.0*I)*y + z,
  x + (-1.0*I)*y - z,
  x + (-1.0*I)*y + (-1.0*I)*z,
  x + (-1.0*I)*y + 1.0*I*z,
  x + 1.0*I*y + (-1.0*I)*z,
  x + 1.0*I*y + 1.0*I*z]]

We see two orbits, the first with 12 bitangents and the second with 16. Their isotropy groups should be index 96/12=8 and 96/16=6, respectively. Indeed we see that we get $C_8$ and $S_3$, respectively.

In [12]:
get_gap_id(isotropy(Fermat_orbits[0],G2))

'[ 8, 1 ]'

In [13]:
get_gap_id(isotropy(Fermat_orbits[1],G2))

'[ 6, 1 ]'

In summary, the bitangents to the Fermat quartic, which has automorphism group $G_2 = C_4^{\times 2}\rtimes S_3$, decompose as a $G_2$-set as

$$[C_4^{\times 2}\rtimes S_3/C_8] + [C_4^{\times 2}\rtimes S_3/S_3].$$

# Type III: The $(48,33)$ quartic<a name="type3"></a>
There is a unique quartic of Type III, defined by the equation
$$x^4 + y^4 + z^4 + (4\rho+2)x^2y^2,$$
where $\rho$ is a primitive 3rd root of unity (note that $4\rho + 2 = 2\sqrt{-3}$, which is another common form for this quartic). Its automorphism group has order 48, and GAP ID `[48,33]`. This type is special in that it is the only kind of symmetric quartic which does not admit a family member defined over $\mathbb{Z}$. To that end, our numerical methods have to be modified a bit - we treat this case at the bottom of the notebook in an "appendix".

# Type IV: The $S_4$ quartic<a name="S4"></a>
For the group $S_4$, we obtain a pencil of *octahedral quartics*, given by the equation
 $$X^4 + Y^4 + Z^4 + a(X^2 Y^2 + Y^2 Z^2 + Z^2 X^2),$$
for any $a\in \mathbb{C}$. Here $S_4$ is subconjugate to $\text{PSL}_2(7)$ as well as $C_4^{\times 2}\rtimes S_3$ inside $\text{PGL}_3(\mathbb{C})$, reflected by certain values of $a$ which allow us to exhibit the Klein and Fermat quartics in this pencil. We pick $a=-3$ (not one of these special values) to carry out our computation:

In [14]:
S4_Quartic = x^4 + y^4 + z^4 -3*(x^2*y^2 + y^2*z^2 + z^2*x^2)

# The automorphism group:
G4gen1=matrix([[0,0,1],[1,0,0],[0,1,0]])
G4gen2=matrix([[0,-1,0],[1,0,0],[0,0,1]])
G4=MatrixGroup(G4gen1,G4gen2)

We end up seeing three orbits, with isotropy groups given by the even and odd copies of $C_2$ in $S_4$, and $S_3$, respectively.

In [15]:
orbs4 = orbits_of_bitangents(S4_Quartic,G4)
H1 = isotropy(orbs4[0],G4); get_gap_id(H1)

'[ 2, 1 ]'

In [16]:
H2 = isotropy(orbs4[1],G4); get_gap_id(H2)

'[ 2, 1 ]'

In [17]:
K = isotropy(orbs4[2],G4); get_gap_id(K)

'[ 6, 1 ]'

We can verify that the two orbits with isotropy $C_2$ are not in bijection as $S_4$-sets, by verifying (calling to GAP) that the two isotropy groups `H1` and `H2` are not conjugate subgroups in `G4`, hence $G4/H1$ and $G4/H2$ are not isomorphic as $G4$-sets:

In [18]:
are_conjugate_subgps(G4,H1,H2)

'false'

Hence we get that the orbits of the 28 bitangents on a Type IV quartic are of the form
$$[S_4/C_2^o] + [S_4/C_2^e] + [S_4/S_3].$$

# Type V: The $(16,13)$ quartic<a name="1613"></a>
For Type V, we obtain a pencil of quartics with equations
$$X^4 + Y^4 + Z^4 + aX^2 Y^2,$$
and symmetry given by the *Pauli group* $P = C_4\circledcirc K_4$. We pick a generic value $a=-4$ to obtain a Type V quartic with no additional symmetry beyond $P$:

In [19]:
# F=QQ; T.<x,y,z>=PolynomialRing(F)
V_Quartic = x^4 + y^4 + z^4 - 4*x^2*y^2

G5gen1 = matrix([
    [-1,0,0],
    [0,1,0],
    [0,0,1]
])
G5gen2 = matrix([
    [E(4),0,0],
    [0,-E(4),0],
    [0,0,1]
])
G5gen3 = matrix([
    [0,-1,0],
    [1,0,0],
    [0,0,1]
])
G5 = MatrixGroup([G5gen1,G5gen2,G5gen3]); #get_gap_id(G5)

We compute that there are exactly four orbits:

In [20]:
orbs5 = orbits_of_bitangents(V_Quartic,G5)
len(orbs5)

4

Let's check which groups are appearing as isotropy groups:

In [21]:
for orb in orbs5:
    print(get_gap_id(isotropy(orb,G5)))

[ 2, 1 ]
[ 2, 1 ]
[ 2, 1 ]
[ 4, 1 ]


We can check that all three of these copies of $C_2$ are not pairwise conjugate!

In [22]:
G5_C2_isotropy_1 = isotropy(orbs5[0],G5)
G5_C2_isotropy_2 = isotropy(orbs5[1],G5)
G5_C2_isotropy_3 = isotropy(orbs5[2],G5)

print(are_conjugate_subgps(G5,G5_C2_isotropy_1,G5_C2_isotropy_2))
print(are_conjugate_subgps(G5,G5_C2_isotropy_1,G5_C2_isotropy_3))
print(are_conjugate_subgps(G5,G5_C2_isotropy_2,G5_C2_isotropy_3))

false
false
false


Altogether we get
$$[P/C_2^{(1)}] + [P/C_2^{(2)}] + [P/C_2^{(3)}] + [P/C_4^Z],$$
where the superscript $Z$ reminds us that the last orbit has *central* isotropy (computation omitted), as there is another non-conjugate copy of $C_4$ living inside of $P$.

# Type VI: The $C_9$ quartic<a name="C9"></a>
There is a unique planar quartic with symmetry group cyclic of order 9, given by the equation
$$X^4 + XY^3 + YZ^3.$$

In [23]:
C9_Quartic = x^4 + x*y^3 + y*z^3
G6 = MatrixGroup([
    matrix([
        [E(9)^3,0,0],
        [0,1,0],
        [0,0,E(9)]
    ])
])


Since the group is cyclic, any subgroup is uniquely determined by its order, so we don't need any calls to GAP, we simply need to list off the lengths of each orbit.

In [24]:
orbs6 = orbits_of_bitangents(C9_Quartic,G6,0.01)

[len(orb) for orb in orbs6]

[9, 9, 9, 1]

Hence the orbits of the bitangents on a $C_9$-symmetric quartic are
$$3[C_9/e] + [C_9/C_9].$$

# Type VII: The $D_8$ quartic<a name="D8"></a>
We have a two-dimensional family of $D_8$-symmetric quartic with equations
$$X^4 + Y^4 + Z^4 + aX^2 Y^2 + bXYZ^2.$$
We pick the special values $a=-3$ and $b=1$ to investigate the action of $D_8$ on the bitangents.

In [25]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
D4_Quartic = x^4 + y^4 + z^4 - 3*x^2*y^2 + x*y*z^2

G7gen1 = matrix([
    [E(4),0,0],
    [0,-E(4),0],
    [0,0,1]
])
G7gen2 = matrix([
    [0,1,0],
    [1,0,0],
    [0,0,1]
])
G7 = MatrixGroup([G7gen1,G7gen2])

The lengths of the orbits are as follows:

In [26]:
[len(orb) for orb in orbits_of_bitangents(D4_Quartic,G7)]

[8, 4, 4, 4, 4, 4]

So we get one transitive orbit $D_8/e$ and four orbits whose isotropy is cyclic of order two. These fall into three conjugacy classes -- two nonconjugate noncentral copies of $C_2$, and one which is the center of $D_8$:

In [27]:
G7_C2_isotropy_1 = isotropy(orbits_of_bitangents(D4_Quartic,G7)[1],G7)
G7_C2_isotropy_2 = isotropy(orbits_of_bitangents(D4_Quartic,G7)[2],G7)
G7_C2_isotropy_3 = isotropy(orbits_of_bitangents(D4_Quartic,G7)[3],G7)
G7_C2_isotropy_4 = isotropy(orbits_of_bitangents(D4_Quartic,G7)[4],G7)
G7_C2_isotropy_5 = isotropy(orbits_of_bitangents(D4_Quartic,G7)[5],G7) # this is the central one

# One pair of orbits
print(are_conjugate_subgps(G7,G7_C2_isotropy_1,G7_C2_isotropy_2))

# Another pair of orbits
print(are_conjugate_subgps(G7,G7_C2_isotropy_3,G7_C2_isotropy_4))

# We can check the fifth one is not conjugate to the other two
print(are_conjugate_subgps(G7,G7_C2_isotropy_5,G7_C2_isotropy_1))
print(are_conjugate_subgps(G7,G7_C2_isotropy_5,G7_C2_isotropy_3))

true
true
false
false


Altogether we get that the bitangents decompose as
$$[D_8/e] + 2[D_8/C_2^{(1)}] + 2[D_8/C_2^{(2)}] + [D_8/C_2^Z].$$

# Type VIII: $C_6$<a name="C6"></a>
There is a pencil of $C_6$-symmetric quartics, given by the equation
$$ x^4 + y^4 + ax^2 y^2 + yz^3$$
We pick the value $a=-3$, and again as $C_6$ is cyclic we only need to know the *sizes* of the orbits:

In [28]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
C6_Quartic = z^3*y +x^4 -3*x^2*y^2 +y^4

G8 = MatrixGroup([
    matrix([
        [-1,0,0],
        [0,1,0],
        [0,0,E(3)]
    ])
])

[len(orb) for orb in orbits_of_bitangents(C6_Quartic,G8)]

[3, 6, 6, 6, 6, 1]

So the bitangents decompose as
$$4[C_6/e] + [C_6/C_2] + [C_6/C_6].$$

# Type IX: The $S_3$ quartic<a name="S3"></a>
We have a 2D family of $S_3$-symmetric quartics, with equations
$$X^3Z + Y^3 Z + X^2 Y^2 + aXYZ^2 + bZ^4.$$
We pick the values $a=-25$ and $b=10$ to compute the bitangents:

In [29]:
S3_Quartic = x^3*z + y^3*z + x^2 * y^2 - 25*x*y*z^2 + 10*z^4
G9gen1 = matrix([
    [E(3),0,0],
    [0,E(3)^2,0],
    [0,0,1]
])
G9gen2 = matrix([
    [0,1,0],
    [1,0,0],
    [0,0,1]
])
G9 = MatrixGroup([G9gen1,G9gen2])

In [30]:
[len(orb) for orb in orbits_of_bitangents(S3_Quartic,G9)]

[6, 6, 3, 1, 6, 3, 3]

It suffices to see if these three orbits with isotropy $C_2$ are in equivariant bijection, and indeed they are:

In [31]:
G9_C2_isotropy_1 = isotropy(orbits_of_bitangents(S3_Quartic,G9)[2],G9)
G9_C2_isotropy_2 = isotropy(orbits_of_bitangents(S3_Quartic,G9)[5],G9)
G9_C2_isotropy_3 = isotropy(orbits_of_bitangents(S3_Quartic,G9)[6],G9)
print(are_conjugate_subgps(G9,G9_C2_isotropy_1,G9_C2_isotropy_2))
print(are_conjugate_subgps(G9,G9_C2_isotropy_1,G9_C2_isotropy_3))

true
true


Altogether we have
$$3[S_3/e] + 3[S_3/C_2] + [S_3/S_3].$$

# Type X: The $K_4$ quartic<a name="K4"></a>
We have a 3D family of $K_4$-symmetric quartics, given by $$X^4 + Y^4 + Z^4 + aX^2 Y^2 + bY^2Z^2 + c Z^2 X^2$$
We pick the special values $(a,b,c) = (-9,-3,-8)$ to compute the bitangents

In [32]:
K4_Quartic = x^4 + y^4 + z^4 -9*x^2*y^2 - 3*y^2*z^2 - 8*x^2*z^2

G10gen1 = matrix([
    [-1,0,0],
    [0,1,0],
    [0,0,1]
])
G10gen2 = matrix([
    [1,0,0],
    [0,-1,0],
    [0,0,1]
])
G10 = MatrixGroup([G10gen1,G10gen2])

[len(orb) for orb in orbits_of_bitangents(K4_Quartic,G10)]

[2, 2, 4, 4, 4, 4, 2, 2, 2, 2]

We get four transitive orbits of the form $[K_4/e]$, and six orbits with isotropy $C_2$. There are three non-conjugate copies of $C_2$ living in $K_4$, and we can verify each of them appears twice (computation omitted, follows via analogous methods to the previous case). Labeling these with an $L$, $R$, and $\Delta$ (left, right, diagonal), we get that the bitangents decompose as
$$4[K_4/e] + 2[K_4/C_2^L] + 2[K_4/C_2^R] + 2[K_4/C_2^\Delta].$$

# Type XI: The $C_3$ quartic<a name="C3"></a>
We have a 3D family of $C_3$-symmetric quartics given by the equations:
$$x(x-y)(x-ay)(x-by) + yz^3.$$
We pick the special values $a=2$ and $b=3$ to compute bitangents. We only need to know sizes of orbits to determine them:

In [33]:
C3_Quartic = z^3*y + x*(x-y)*(x-2*y)*(x-3*y)

G11 = MatrixGroup([
    matrix([
        [1,0,0],
        [0,1,0],
        [0,0,E(3)]
    ])
])

[len(orb) for orb in orbits_of_bitangents(C3_Quartic,G11)]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 1]

So we get that the bitangents decompose as
$$9[C_3/e] + [C_3/C_3].$$

# Type XII: The $C_2$ quartic
Finally we have a 4D family of $C_2$-symmetric quartics, given by the equations
$$x^4 + x^2 L_2 + L_4,$$
where $L_n$ is any binary form of degree $n$ in $y$ and $z$. Here the $C_2$-action comes from sending $x\mapsto -x$. We can easily see the action on bitangents by swapping signs, so we don't need the full power of our code, but we use it anyways. We pick the special values
$$L_2 = -y^2 - 4z^2\quad \text{ and } \quad L_4 = y^4 - y^2 z^2 - z^4$$

In [34]:
L2 = -y^2 - 4*z^2
L4 = y^4 - y^2*z^2 - z^4
C2_Quartic = x^4 + x^2*L2 + L4

G12 = MatrixGroup([
    matrix([
        [-1,0,0],
        [0,1,0],
        [0,0,1]
    ])
])
[len(orb) for orb in orbits_of_bitangents(C2_Quartic,G12)]

orbits_of_bitangents(C2_Quartic,G12)

[[y + (-2.2948)*z],
 [y + 2.2948*z],
 [y + (-1.1252*I)*z],
 [y + 1.1252*I*z],
 [x + (0.9682 + 0.25*I)*y + (-0.9937 - 0.4872*I)*z,
  x + (-0.9682 - 0.25*I)*y + (0.9937 + 0.4872*I)*z],
 [x + (0.9682 - 0.25*I)*y + (-0.9937 + 0.4872*I)*z,
  x + (-0.9682 + 0.25*I)*y + (0.9937 - 0.4872*I)*z],
 [x + (-0.7906 - 0.6124*I)*y + (-0.3978 + 0.8113*I)*z,
  x + (0.7906 + 0.6124*I)*y + (0.3978 - 0.8113*I)*z],
 [x + (-0.7906 + 0.6124*I)*y + (-0.3978 - 0.8113*I)*z,
  x + (0.7906 - 0.6124*I)*y + (0.3978 + 0.8113*I)*z],
 [x + (0.7906 - 0.6124*I)*y + (-0.3978 - 0.8113*I)*z,
  x + (-0.7906 + 0.6124*I)*y + (0.3978 + 0.8113*I)*z],
 [x + (0.7906 + 0.6124*I)*y + (-0.3978 + 0.8113*I)*z,
  x + (-0.7906 - 0.6124*I)*y + (0.3978 - 0.8113*I)*z],
 [x + (-0.9682 - 0.25*I)*y + (-0.9937 - 0.4872*I)*z,
  x + (0.9682 + 0.25*I)*y + (0.9937 + 0.4872*I)*z],
 [x + (-0.9682 + 0.25*I)*y + (-0.9937 + 0.4872*I)*z,
  x + (0.9682 - 0.25*I)*y + (0.9937 - 0.4872*I)*z],
 [x + (-2.5032)*z, x + 2.5032*z],
 [x + (-0.5157*I)*z, x + 0.5157*

Hence we have four fixed bitangents (these are horizontal lines on the $z=1$ affine patch), and the rest come in pairs:
$$12[C_2/e] + 4[C_2/C_2].$$

## Appendix: Type III in detail
Recall we are considering the Type III quartic
$$x^4 + y^4 + z^4 + (4\rho+2)x^2y^2,$$
where $\rho$ is a primitive 3rd root of unity. We would like to compute equations for its bitangents, however since $\rho\notin\mathbb{Q}$, we must modify the function `compute_bitangents`. To that end we include the following modified method:

In [35]:
def compute_bitangents_ideal_Q3(f):
    F=CyclotomicField(3); T.<x,y,z>=PolynomialRing(F)
    # Check whether the quartic is smooth
    # Grad=ideal(f,diff(f,x),diff(f,y),diff(f,z))
    # if not Grad.dimension()==0:
    #     sys.exit("Quartic is not smooth!")
    f = T(f)
    R.<x,y,z,a,b,a0,a1,a2,a3,a4>=PolynomialRing(F)
    f0=f.base_extend(R)
    S.<a,b>=PolynomialRing(F)
    digits=50
    threshold=0.000000000001
    almostzero=threshold

    # Make a formal line
    Line= a*x+b*y+z;
    
    puresquare=ideal(a0*a3^2-a1^2*a4,8*a0^2*a3-4*a0*a1*a2+a1^3,8*a1*a4^2-4*a2*a3*a4+a3^3,8*a0*a1*a4-4*a0*a2*a3+a1^2*a3,8*a0*a3*a4-4*a1*a2*a4+a1*a3^2,16*a0^2*a4+2*a0*a1*a3-4*a0*a2^2+a1^2*a2,16*a0*a4^2+2*a1*a3*a4-4*a2^2*a4+a2*a3^2);
    Res=f0.resultant(Line,z)    
    Res=Res.subs(y=1)    
    phi=hom(R,S,[0,0,0,a,b,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])
    bit1 = phi(puresquare)    
    
    I_ideal=singular.groebner(singular(bit1))
    return I_ideal

Dolg = x^4 + y^4 + z^4 + 2*sqrt(-3)*(x^2)*(y^2)
F=CyclotomicField(12); T.<x,y,z>=PolynomialRing(F)

# Grad = ideal(Dolg,diff(Dolg,x),diff(Dolg,y),diff(Dolg,z))
Iideal = compute_bitangents_ideal_Q3(Dolg);Iideal
print(Iideal)

a^4+(-4*zeta3-2)*a^2*b^2+b^4+4,
3*a^3*b^3+(2*zeta3+1)*a*b^5+(4*zeta3+2)*a*b,
7*a^2*b^5+(2*zeta3+1)*b^7+2*a^2*b+(8*zeta3+4)*b^3,
2*a*b^7+(2*zeta3+1)*a^3*b+a*b^3,
2*b^9-5*a^4*b+(-6*zeta3-3)*a^2*b^3+10*b^5+8*b


This gives us an ideal in a polynomial ring over $\mathbb{Q}(\zeta_3)$ that we wish to solve -- the solutions will be bitangents of the form $ax+by+z=0$.

In [36]:
# Grad.dimension()
# cyc

R.<a,b> =PolynomialRing(CC)
rho = exp(2*pi*i/3)
a,b = var('a b')
f0 = a^4+(-4*rho-2)*a^2*b^2+b^4+4
f1 = 3*a^3*b^3+(2*rho+1)*a*b^5+(4*rho+2)*a*b
f2 = 7*a^2*b^5+(2*rho+1)*b^7+2*a^2*b+(8*rho+4)*b^3
f3 = 2*a*b^7+(2*rho+1)*a^3*b+a*b^3
f4 = 2*b^9-5*a^4*b+(-6*rho-3)*a^2*b^3+10*b^5+8*b


candidates = solve([f1==0,f2==0,f3==0,f0==0,f4==0],a,b); candidates

[[a == -1/2*I*sqrt(3) - 1/2, b == -1/2*I*sqrt(3) - 1/2],
 [a == 1/2*I*sqrt(3) + 1/2, b == -1/2*I*sqrt(3) - 1/2],
 [a == -1/2*sqrt(3) - 1/2*I, b == 1/2*I*sqrt(3) - 1/2],
 [a == 1/2*sqrt(3) + 1/2*I, b == 1/2*I*sqrt(3) - 1/2],
 [a == -1/2*sqrt(3) - 1/2*I, b == -1/2*I*sqrt(3) + 1/2],
 [a == 1/2*sqrt(3) + 1/2*I, b == -1/2*I*sqrt(3) + 1/2],
 [a == -1/2*I*sqrt(3) - 1/2, b == 1/2*I*sqrt(3) + 1/2],
 [a == 1/2*I*sqrt(3) + 1/2, b == 1/2*I*sqrt(3) + 1/2],
 [a == 1/2*I*sqrt(3) - 1/2, b == -1/2*sqrt(3) - 1/2*I],
 [a == -1/2*I*sqrt(3) + 1/2, b == -1/2*sqrt(3) - 1/2*I],
 [a == 1/2*I*sqrt(3) - 1/2, b == 1/2*sqrt(3) + 1/2*I],
 [a == -1/2*I*sqrt(3) + 1/2, b == 1/2*sqrt(3) + 1/2*I],
 [a == -1/2*sqrt(3) + 1/2*I, b == -1/2*sqrt(3) + 1/2*I],
 [a == 1/2*sqrt(3) - 1/2*I, b == -1/2*sqrt(3) + 1/2*I],
 [a == -1/2*sqrt(3) + 1/2*I, b == 1/2*sqrt(3) - 1/2*I],
 [a == 1/2*sqrt(3) - 1/2*I, b == 1/2*sqrt(3) - 1/2*I],
 [a == 0, b == (-I + 1)],
 [a == 0, b == (I + 1)],
 [a == 0, b == (-I - 1)],
 [a == 0, b == (I - 1)],
 [

This gives us 24 bitangents. To obtain the other 4, we need to consider the case where $z=0$. Altogether we get the following

In [37]:
QuarticIIIBis = [x + (-0.50000084 + 0.50000314*I)*z,
x + (-0.50000314 - 0.50000084*I)*z,
x - y + (0.500000000000000 - 0.866030000000000*I)*z,
x - y + (-0.500000000000000 + 0.866030000000000*I)*z,
x - y + (-0.866030000000000 - 0.500000000000000*I)*z,
x - y + (0.866030000000000 + 0.500000000000000*I)*z,
x + (0.50000314 + 0.50000084*I)*z,
x + (0.50000084 - 0.50000314*I)*z,
y + (0.50000314 + 0.50000084*I)*z,
y + (-0.50000084 + 0.50000314*I)*z,
y + (0.50000084 - 0.50000314*I)*z,
y + (-0.50000314 - 0.50000084*I)*z,
x + 1.00000000000000*I*y + (0.500000000000000 + 0.866030000000000*I)*z,
x + 1.00000000000000*I*y + (0.866030000000000 - 0.500000000000000*I)*z,
x + 1.00000000000000*I*y + (-0.866030000000000 + 0.500000000000000*I)*z,
x + 1.00000000000000*I*y + (-0.500000000000000 - 0.866030000000000*I)*z,
x + (-1.00000000000000*I)*y + (0.500000000000000 + 0.866030000000000*I)*z,
x + (-1.00000000000000*I)*y + (0.866030000000000 - 0.500000000000000*I)*z,
x + (-1.00000000000000*I)*y + (-0.866030000000000 + 0.500000000000000*I)*z,
x + (-1.00000000000000*I)*y + (-0.500000000000000 - 0.866030000000000*I)*z,
x + y + (0.500000000000000 - 0.866030000000000*I)*z,
x + y + (-0.866030000000000 - 0.500000000000000*I)*z,
x + y + (-0.500000000000000 + 0.866030000000000*I)*z,
x + y + (0.866030000000000 + 0.500000000000000*I)*z,
x + (-0.36602540 - 0.36602540*I)*y,
x + (0.36602540 + 0.36602540*I)*y,
x + (-1.3660254 + 1.3660254*I)*y,
x + (1.3660254 - 1.3660254*I)*y]

We can now work through the action of the automorphism group explicitly on these bitangents to compute the decomposition (omitted, can add if requested).